In [1]:
# if os.name != 'posix':
#     %matplotlib ipympl
# else:
#     %matplotlib notebook

import numpy as np
import cmath
import matplotlib.pyplot as plt
import scipy.constants as sc
# plt.rcParams.update({
#     "text.usetex": True,
#     'text.latex.preamble': r"\usepackage{amsmath} \boldmath"
# })
import os
import h5py

print(os.getpid())
%cd ../../

import pylib.mix as mix
import pylib.measurement as mse

15267
/media/work/docs/codes/QuCF/scripts-py


In [18]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info.operators import Operator
from qiskit.compiler import transpile
from scipy.linalg import expm

# --- Create a diagonal matrix with x-coordinates on the main diagonal ---
nq_ = 5;
t_ = 2.0;
N_ = 1 << nq_
x_min, x_max = 0.0, np.pi
# x_min, x_max = -1.0, 1.0
x = np.linspace(x_min, x_max, N_)
A = np.diag(x)


# # --- Create the quantum circuit ---
# Uexp = expm(-1.j*t_*A)

# q = QuantumRegister(nq_, "r")
# qc = QuantumCircuit(q)

# tqs = []
# for ii in range(nq_):
#     tqs.append(q[ii])

# circU = Operator( Uexp )
# qc.unitary(circU, tqs, label="exp")
# qc.draw(output='mpl')

# # --- Transpile the circuit ---
# new_qc = transpile(qc, basis_gates = ['rz', 'rx', 'cx'], optimization_level = 3)
# print("nq, circuit depth: {:d}, {:d}".format(nq_, new_qc.depth()))
# # new_qc.draw()

In [19]:
# ----------------------------------------------------------------------------
# --- Decompose the diagonal matrix into a tensor product of Z-Pauli gates ---
# ----------------------------------------------------------------------------
mix.reload_module(mix)

import itertools as it
A_dec = mix.hermitian_to_Z_pauli(A, flag_filter = True, small_coef = 1e-14, flag_print_details=True)
# A_dec = mix.hermitian_to_pauli(A, flag_filter = True, small_coef = 1e-14, flag_print_details=True)

---------------------------------------------------------------
--- Decomposition of a Hermitian into Pauli tensor products ---
nq:  5
Total number of possible combinations of (I,Z) Pauli matrices 32
n. of Pauli products:  6
---------------------------------------------------------------


In [20]:
# -------------------------------------------------
# --- Form a circuit to model exp(-1j*t_*A_dec) ---
# -------------------------------------------------
import pylib.QC as QC

# --- Construction of the circuit ---
qc_dec = QC.Circuit(nq_)
qc_dec.form_from_pauli(A_dec, t_, sel_error_order = 2)
print(qc_dec.form_qiskit_circuit().draw(fold = 160)); print("")
Ures = qc_dec.get_matrix()

# --- Recheck the construction ---
max_abs_err = 0.0
for id_x in range(N_):
    init_state = np.zeros(N_)
    init_state[id_x] = 1

    res_U = Ures.dot(init_state)[0,id_x]
    res_dir_exp = np.exp(-1.j*t_*x[id_x]) 
    print("{:0.3f}, {:0.3f}".format(res_U.real, res_dir_exp.real))

    abs_err = np.abs(res_U - res_dir_exp)
    if abs_err > max_abs_err:
        max_abs_err = abs_err

print("max. abs. diff: {:0.3e}".format(max_abs_err))

     ┌─────────────┐ 
q_0: ┤ Rz(-3.2429) ├─
     ├─────────────┤ 
q_1: ┤ Rz(-1.6215) ├─
     ├─────────────┴┐
q_2: ┤ Rz(-0.81073) ├
     ├──────────────┤
q_3: ┤ Rz(-0.40537) ├
     ├──────────────┤
q_4: ┤ Rz(-0.20268) ├
     └──────────────┘

-1.000, 1.000
-0.980, 0.980
-0.919, 0.919
-0.821, 0.821
-0.689, 0.689
-0.529, 0.529
-0.347, 0.347
-0.151, 0.151
0.051, -0.051
0.251, -0.251
0.440, -0.440
0.612, -0.612
0.759, -0.759
0.874, -0.874
0.954, -0.954
0.995, -0.995
0.995, -0.995
0.954, -0.954
0.874, -0.874
0.759, -0.759
0.612, -0.612
0.440, -0.440
0.251, -0.251
0.051, -0.051
-0.151, 0.151
-0.347, 0.347
-0.529, 0.529
-0.689, 0.689
-0.821, 0.821
-0.919, 0.919
-0.980, 0.980
-1.000, 1.000
max. abs. diff: 2.000e+00


In [10]:
for ii in range(len(A_dec)):
    print(A_dec[ii][0])

1.5707963267948966
-0.05067084925144827
-0.10134169850289654
-0.2026833970057932
-0.4053667940115862
-0.8107335880231725


In [8]:
np.pi/2.

1.5707963267948966

In [18]:
qc_dec